# New York Airbnb Analytics

### Data Processing For Group Project

# Setup

In [2]:

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

#Suppress all warning
import warnings
warnings.filterwarnings("ignore")
# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

#Display multiple outputs per cell

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# Get the data

In [3]:
import pandas as pd
df1=pd.read_csv('http://data.insideairbnb.com/united-states/ny/new-york-city/2021-02-04/data/listings.csv.gz')
df1.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,20210204180331,2021-02-05,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",Centrally located in the heart of Manhattan ju...,https://a0.muscache.com/pictures/f0813a11-40b2...,2845,https://www.airbnb.com/users/show/2845,...,10.0,10.0,9.0,NaN,f,2,2,0,0,0.35
1,3831,https://www.airbnb.com/rooms/3831,20210204180331,2021-02-05,"Whole flr w/private bdrm, bath & kitchen(pls r...","Enjoy 500 s.f. top floor in 1899 brownstone, w...",Just the right mix of urban center and local n...,https://a0.muscache.com/pictures/e49999c2-9fd5...,4869,https://www.airbnb.com/users/show/4869,...,10.0,10.0,10.0,NaN,f,1,1,0,0,4.99
2,5121,https://www.airbnb.com/rooms/5121,20210204180331,2021-02-05,BlissArtsSpace!,<b>The space</b><br />HELLO EVERYONE AND THANK...,NaN,https://a0.muscache.com/pictures/2090980c-b68e...,7356,https://www.airbnb.com/users/show/7356,...,10.0,9.0,9.0,NaN,f,1,0,1,0,0.35
3,5178,https://www.airbnb.com/rooms/5178,20210204180331,2021-02-05,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,"Theater district, many restaurants around here.",https://a0.muscache.com/pictures/12065/f070997...,8967,https://www.airbnb.com/users/show/8967,...,9.0,10.0,9.0,NaN,f,1,0,1,0,3.31
4,5203,https://www.airbnb.com/rooms/5203,20210204180331,2021-02-06,Cozy Clean Guest Room - Family Apt,"Our best guests are seeking a safe, clean, spa...",Our neighborhood is full of restaurants and ca...,https://a0.muscache.com/pictures/103776/b37157...,7490,https://www.airbnb.com/users/show/7490,...,10.0,10.0,10.0,NaN,f,1,0,1,0,0.85


- Pick the valuable columns (not the columns with unique value for each row)

In [4]:
df1.drop(df1.columns[np.r_[0:3,6:10,12,18:22,24,26,34,31,42:55,56,61:68,69:73]], axis=1, inplace=True)
df1.head()

corr_matrix=df1.corr()
corr_matrix["review_scores_rating"].sort_values(ascending=False)

,last_scraped,name,description,host_name,host_since,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,...,price,minimum_nights,maximum_nights,number_of_reviews,number_of_reviews_l30d,first_review,last_review,review_scores_rating,instant_bookable,reviews_per_month
0,2021-02-05,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",Jennifer,2008-09-09,A New Yorker since 2000! My passion is creatin...,within a few hours,93%,26%,f,...,$100.00,30,1125,48,0,2009-11-21,2019-11-04,94.0,f,0.35
1,2021-02-05,"Whole flr w/private bdrm, bath & kitchen(pls r...","Enjoy 500 s.f. top floor in 1899 brownstone, w...",LisaRoxanne,2008-12-07,Laid-back Native New Yorker (formerly bi-coast...,within a few hours,98%,93%,f,...,$73.00,1,730,386,3,2014-09-30,2021-01-27,90.0,f,4.99
2,2021-02-05,BlissArtsSpace!,<b>The space</b><br />HELLO EVERYONE AND THANK...,Garon,2009-02-03,"I am an artist(painter, filmmaker) and curato...",NaN,NaN,NaN,f,...,$60.00,30,730,50,0,2009-05-28,2019-12-02,90.0,f,0.35
3,2021-02-05,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,Shunichi,2009-03-03,I used to work for a financial industry but no...,within a day,100%,100%,f,...,$79.00,2,14,474,0,2009-05-06,2020-09-25,84.0,f,3.31
4,2021-02-06,Cozy Clean Guest Room - Family Apt,"Our best guests are seeking a safe, clean, spa...",MaryEllen,2009-02-05,Welcome to family life with my oldest two away...,NaN,NaN,NaN,f,...,$75.00,2,14,118,0,2009-09-07,2017-07-21,98.0,f,0.85


review_scores_rating         1.000000
number_of_reviews            0.054595
reviews_per_month            0.041548
bedrooms                     0.010334
number_of_reviews_l30d       0.005152
accommodates                -0.006905
beds                        -0.011301
maximum_nights              -0.013451
longitude                   -0.015705
minimum_nights              -0.025785
latitude                    -0.031618
host_total_listings_count   -0.032321
Name: review_scores_rating, dtype: float64

### 2. Clean/ Processing the dataset

In [5]:
#=======================================================Data Cleansing===================================================
#Only pick the Airbnb apartment with reviews
df1=df1[df1['number_of_reviews'].astype(int)>0]
#Only pick the Airbnb apartment with price
df1=df1[df1['price'].apply(lambda x: len(x)>0)]
#Only pick the Airbnb with the answer(t/f) for "host_is_superhost"
df1=df1[df1['host_is_superhost'].apply(lambda x: len(str(x))==1)]
#Drop other answer except f/t in "instant_bookable"
df1=df1[df1['instant_bookable'].isin(['f','t'])]
#To drop the review score which lower than 21 (potential outliers)
df1=df1[df1['review_scores_rating']>21]
#Replace all the blank cell with NaN value
df1=df1.replace('',np.NaN)
#========================================================Add new features===============================================
#Get the length of the sentence in following five columns (number of words)
df1['name_length']  = df1['name'].apply(lambda x: len(str(x).split()))
df1['description_length']=df1['description'].apply(lambda x: len(str(x).split()))
df1['host_about_length']=df1['host_about'].apply(lambda x: len(str(x).split()))
df1['verifications_length']=df1['host_verifications'].apply(lambda x: len(str(x).split(',')))
df1['amenities_length']=df1['amenities'].apply(lambda x: len(str(x).split(',')))
#Get the difference between the "last_scraped" with the following dates (in days)
df1['host_since_days'] = (pd.to_datetime(df1['last_scraped'])-pd.to_datetime(df1['host_since'])).dt.days
df1['first_reviews_days'] = (pd.to_datetime(df1['last_scraped'])-pd.to_datetime(df1['first_review'])).dt.days
df1['last_review_days'] = (pd.to_datetime(df1['last_scraped'])-pd.to_datetime(df1['last_review'])).dt.days

#Convert price column to the float datatype
df1['price'] = df1['price'].str.replace(',', '').str.replace('$', '').astype(float)
#Get the new column to express the price per accommodate
df1['price_per_accommodates']=df1['price']/df1['accommodates'].astype(float)
#=====================================================Change the data type=============================================
#Convert the categorical columns to dummified columns
list_col=['host_is_superhost','host_identity_verified','instant_bookable']
for i in list_col:
    df1[i] = np.where(df1[i]== 't', 1, 0)
    
#Add the dummified column by extract the word from "amenities" column
df1['has_Hair_dryer']=np.where(df1['amenities'].str.contains('Hair dryer'), 1, 0)
#df[df['date'].astype(str).str.contains('07311954')]

- Apply impute package to fill the missing data in numerical columns

In [6]:
print(df1.shape)

df1.info()

(26633, 43)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 26633 entries, 0 to 36915
Data columns (total 43 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   last_scraped                  26633 non-null  object 
 1   name                          26628 non-null  object 
 2   description                   26174 non-null  object 
 3   host_name                     26633 non-null  object 
 4   host_since                    26633 non-null  object 
 5   host_about                    17074 non-null  object 
 6   host_response_time            13574 non-null  object 
 7   host_response_rate            13574 non-null  object 
 8   host_acceptance_rate          17229 non-null  object 
 9   host_is_superhost             26633 non-null  int32  
 10  host_total_listings_count     26633 non-null  float64
 11  host_verifications            26633 non-null  object 
 12  host_identity_verified        26633 non-null  in

### Check for missing values

In [7]:
def missing_values(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(columns={
        0: 'Missing Values',
        1: '% of Total Values'
    })
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
            '% of Total Values', ascending=False).round(1)
    print("Dataframe has " + str(df.shape[1]) + " columns.")
    print("There are " + str(mis_val_table_ren_columns.shape[0]) +
          " columns that have missing values.")
    
    return mis_val_table_ren_columns

# Missing values statistics
miss_values = missing_values(df1)
miss_values.head(20)

Dataframe has 43 columns.
There are 9 columns that have missing values.


,Missing Values,% of Total Values
host_response_time,13059,49.0
host_response_rate,13059,49.0
host_about,9559,35.9
host_acceptance_rate,9404,35.3
bedrooms,2522,9.5
description,459,1.7
beds,130,0.5
bathrooms_text,39,0.1
name,5,0.0


### Flag Missing Values

**Create a new column called 'Flag' which is 1 if a row is complete with all its values, and 0 if there is a missing value in the row**

In [8]:
df1['flag'] = df1.notna().all(axis=1).view('i1')

df1

,last_scraped,name,description,host_name,host_since,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,...,description_length,host_about_length,verifications_length,amenities_length,host_since_days,first_reviews_days,last_review_days,price_per_accommodates,has_Hair_dryer,flag
0,2021-02-05,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",Jennifer,2008-09-09,A New Yorker since 2000! My passion is creatin...,within a few hours,93%,26%,0,...,151,67,9,31,4532,4094,459,50.000000,1,0
1,2021-02-05,"Whole flr w/private bdrm, bath & kitchen(pls r...","Enjoy 500 s.f. top floor in 1899 brownstone, w...",LisaRoxanne,2008-12-07,Laid-back Native New Yorker (formerly bi-coast...,within a few hours,98%,93%,0,...,165,35,6,35,4443,2320,9,24.333333,1,1
2,2021-02-05,BlissArtsSpace!,<b>The space</b><br />HELLO EVERYONE AND THANK...,Garon,2009-02-03,"I am an artist(painter, filmmaker) and curato...",NaN,NaN,NaN,0,...,175,69,8,5,4385,4271,431,30.000000,0,0
3,2021-02-05,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,Shunichi,2009-03-03,I used to work for a financial industry but no...,within a day,100%,100%,0,...,165,21,4,18,4357,4293,133,39.500000,1,1
4,2021-02-06,Cozy Clean Guest Room - Family Apt,"Our best guests are seeking a safe, clean, spa...",MaryEllen,2009-02-05,Welcome to family life with my oldest two away...,NaN,NaN,NaN,0,...,171,140,7,16,4384,4170,1296,75.000000,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36818,2021-02-06,lanigirO JFK•Casino•Beach,This is an entire compact basement studio to y...,Ryan,2017-06-03,NaN,within an hour,100%,98%,1,...,168,1,10,16,1344,1,1,55.000000,0,0
36832,2021-02-05,Beautiful Room at Central Park West,NaN,Deniz,2015-08-03,I am Deniz 34 years old student pilot in New Y...,within an hour,100%,100%,0,...,1,23,5,20,2013,6,6,46.000000,1,0
36848,2021-02-05,Big Blue Room & confortable in Brooklyn,"It is a beautiful private room, the bathroom i...",Cristina And Sasha,2016-11-15,Sasha is from Omaha originally but has lived i...,within an hour,100%,98%,0,...,146,221,7,28,1543,1,1,32.500000,1,1
36851,2021-02-05,Cozy room in heart of Brooklyn💓😌,NaN,Valentina,2019-10-07,NaN,within a few hours,91%,100%,0,...,1,1,6,20,487,5,5,15.000000,1,0


In [9]:
sum(df1.flag)

7831

There are 7831 rows with at least 1 missing value

**Another flagging method: flag the missing rows in each column separately**

In [10]:
cols = ['host_response_time','host_response_rate','host_acceptance_rate','bedrooms','description','beds','bathrooms_text','name']
df_try = df1[cols].isnull().astype(int).add_suffix('_indicator')
#df_try

#merge both the df1 and the flagged columns
df2 = pd.merge(df1, df_try, left_index=True, right_index=True)
df2.head(10)

,last_scraped,name,description,host_name,host_since,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,...,has_Hair_dryer,flag,host_response_time_indicator,host_response_rate_indicator,host_acceptance_rate_indicator,bedrooms_indicator,description_indicator,beds_indicator,bathrooms_text_indicator,name_indicator
0,2021-02-05,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",Jennifer,2008-09-09,A New Yorker since 2000! My passion is creatin...,within a few hours,93%,26%,0,...,1,0,0,0,0,1,0,0,0,0
1,2021-02-05,"Whole flr w/private bdrm, bath & kitchen(pls r...","Enjoy 500 s.f. top floor in 1899 brownstone, w...",LisaRoxanne,2008-12-07,Laid-back Native New Yorker (formerly bi-coast...,within a few hours,98%,93%,0,...,1,1,0,0,0,0,0,0,0,0
2,2021-02-05,BlissArtsSpace!,<b>The space</b><br />HELLO EVERYONE AND THANK...,Garon,2009-02-03,"I am an artist(painter, filmmaker) and curato...",NaN,NaN,NaN,0,...,0,0,1,1,1,0,0,0,1,0
3,2021-02-05,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,Shunichi,2009-03-03,I used to work for a financial industry but no...,within a day,100%,100%,0,...,1,1,0,0,0,0,0,0,0,0
4,2021-02-06,Cozy Clean Guest Room - Family Apt,"Our best guests are seeking a safe, clean, spa...",MaryEllen,2009-02-05,Welcome to family life with my oldest two away...,NaN,NaN,NaN,0,...,1,0,1,1,1,0,0,0,0,0
5,2021-02-05,"Lovely Room 1, Garden, Best Area, Legal rental","Discounted now! Beautiful house, gorgeous ga...",Laurie,2009-03-10,"Hello, \r\nI will be welcoming and helpful, w...",within an hour,100%,100%,1,...,1,1,0,0,0,0,0,0,0,0
6,2021-02-05,Only 2 stops to Manhattan studio,Comfortable studio apartment with super comfor...,Allen & Irina,2009-05-06,We love to travel. When we travel we like to s...,NaN,NaN,75%,0,...,1,0,1,1,0,1,0,0,0,0
8,2021-02-06,UES Beautiful Blue Room,Beautiful peaceful healthy home<br /><br /><b>...,Cyn,2009-05-12,Capturing the Steinbeck side of life in its Fi...,within an hour,100%,100%,0,...,1,1,0,0,0,0,0,0,0,0
9,2021-02-05,Perfect for Your Parents: Privacy + Garden,"Parents/grandparents coming to town, or are yo...",Jane,2009-05-17,I have been an Airbnb host since 2009 -- just ...,within an hour,100%,100%,1,...,1,1,0,0,0,0,0,0,0,0
11,2021-02-06,Sweet and Spacious Brooklyn Loft,A true open-plan loft in a repurposed factory ...,Chaya,2009-06-12,We're a couple in our thirties who love to tra...,within an hour,100%,62%,0,...,0,0,0,0,0,1,0,0,0,0


**Convert 'host_acceptance_rate' and 'host_response_rate' to numerical variables by removing the %'**

In [11]:
#get all the non-null values and Convert the object datatype to numerical datatype: host_response_rate, host_acceptance_rate
test = df2[['host_response_rate','host_acceptance_rate']].dropna()
test['host_acceptance_rate'] =test['host_acceptance_rate'].str.replace('%',"").astype(int)/100
test['host_response_rate'] =test['host_response_rate'].str.replace('%',"").astype(int)/100
test

#Add the numeric values back into the dataframe (replacement)
df2['host_response_rate'].loc[test.index] = test['host_response_rate']
df2['host_acceptance_rate'].loc[test.index] = test['host_acceptance_rate']
df2

,host_response_rate,host_acceptance_rate
0,0.93,0.26
1,0.98,0.93
3,1.00,1.00
5,1.00,1.00
8,1.00,1.00
...,...,...
36818,1.00,0.98
36832,1.00,1.00
36848,1.00,0.98
36851,0.91,1.00


,last_scraped,name,description,host_name,host_since,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,...,has_Hair_dryer,flag,host_response_time_indicator,host_response_rate_indicator,host_acceptance_rate_indicator,bedrooms_indicator,description_indicator,beds_indicator,bathrooms_text_indicator,name_indicator
0,2021-02-05,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",Jennifer,2008-09-09,A New Yorker since 2000! My passion is creatin...,within a few hours,0.93,0.26,0,...,1,0,0,0,0,1,0,0,0,0
1,2021-02-05,"Whole flr w/private bdrm, bath & kitchen(pls r...","Enjoy 500 s.f. top floor in 1899 brownstone, w...",LisaRoxanne,2008-12-07,Laid-back Native New Yorker (formerly bi-coast...,within a few hours,0.98,0.93,0,...,1,1,0,0,0,0,0,0,0,0
2,2021-02-05,BlissArtsSpace!,<b>The space</b><br />HELLO EVERYONE AND THANK...,Garon,2009-02-03,"I am an artist(painter, filmmaker) and curato...",NaN,NaN,NaN,0,...,0,0,1,1,1,0,0,0,1,0
3,2021-02-05,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,Shunichi,2009-03-03,I used to work for a financial industry but no...,within a day,1,1,0,...,1,1,0,0,0,0,0,0,0,0
4,2021-02-06,Cozy Clean Guest Room - Family Apt,"Our best guests are seeking a safe, clean, spa...",MaryEllen,2009-02-05,Welcome to family life with my oldest two away...,NaN,NaN,NaN,0,...,1,0,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36818,2021-02-06,lanigirO JFK•Casino•Beach,This is an entire compact basement studio to y...,Ryan,2017-06-03,NaN,within an hour,1,0.98,1,...,0,0,0,0,0,0,0,0,0,0
36832,2021-02-05,Beautiful Room at Central Park West,NaN,Deniz,2015-08-03,I am Deniz 34 years old student pilot in New Y...,within an hour,1,1,0,...,1,0,0,0,0,0,1,0,0,0
36848,2021-02-05,Big Blue Room & confortable in Brooklyn,"It is a beautiful private room, the bathroom i...",Cristina And Sasha,2016-11-15,Sasha is from Omaha originally but has lived i...,within an hour,1,0.98,0,...,1,1,0,0,0,0,0,0,0,0
36851,2021-02-05,Cozy room in heart of Brooklyn💓😌,NaN,Valentina,2019-10-07,NaN,within a few hours,0.91,1,0,...,1,0,0,0,0,0,1,0,0,0


### Iterative imputer on numerical variables

In [12]:
#for some reason, the added values become 'objects', so you need to convert them to 'floats'
df2['host_response_rate'] = pd.to_numeric(df2['host_response_rate'],errors = 'coerce')
df2['host_acceptance_rate'] = pd.to_numeric(df2['host_acceptance_rate'],errors = 'coerce')

#choose numerical variables only
df_num=df2.drop(['neighbourhood_group_cleansed'],axis=1).iloc[:,np.r_[7:11,12,14:16,17,19:21,22:27,29:42]]

df_num

# Missing values statistics
miss_values = missing_values(df_num)
miss_values.head(20)

,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_identity_verified,latitude,longitude,accommodates,bedrooms,beds,...,name_length,description_length,host_about_length,verifications_length,amenities_length,host_since_days,first_reviews_days,last_review_days,price_per_accommodates,has_Hair_dryer
0,0.93,0.26,0,6.0,1,40.75362,-73.98377,2,NaN,1.0,...,3,151,67,9,31,4532,4094,459,50.000000,1
1,0.98,0.93,0,1.0,1,40.68514,-73.95976,3,1.0,3.0,...,8,165,35,6,35,4443,2320,9,24.333333,1
2,NaN,NaN,0,1.0,1,40.68688,-73.95596,2,1.0,1.0,...,1,175,69,8,5,4385,4271,431,30.000000,0
3,1.00,1.00,0,1.0,0,40.76468,-73.98315,2,1.0,1.0,...,5,165,21,4,18,4357,4293,133,39.500000,1
4,NaN,NaN,0,1.0,1,40.80178,-73.96723,1,1.0,1.0,...,7,171,140,7,16,4384,4170,1296,75.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36818,1.00,0.98,1,3.0,1,40.67319,-73.84068,2,1.0,1.0,...,2,168,1,10,16,1344,1,1,55.000000,0
36832,1.00,1.00,0,4.0,1,40.79851,-73.96004,1,4.0,5.0,...,6,1,23,5,20,2013,6,6,46.000000,1
36848,1.00,0.98,0,4.0,1,40.70146,-73.94368,2,1.0,1.0,...,7,146,221,7,28,1543,1,1,32.500000,1
36851,0.91,1.00,0,1.0,1,40.70430,-73.92561,2,1.0,1.0,...,6,1,1,6,20,487,5,5,15.000000,1


Dataframe has 28 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
host_response_rate,13487,50.6
host_acceptance_rate,13487,50.6
bedrooms,2522,9.5
beds,130,0.5


In [13]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp = IterativeImputer(random_state=0)

df_num1 = imp.fit_transform(df_num)

cols = list(df_num)

df_num1=pd.DataFrame(df_num1)
df_num1.columns=cols
df_num1

# Re-check Missing values statistics
miss_values = missing_values(df_num1)
miss_values.head(20)

,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_identity_verified,latitude,longitude,accommodates,bedrooms,beds,...,name_length,description_length,host_about_length,verifications_length,amenities_length,host_since_days,first_reviews_days,last_review_days,price_per_accommodates,has_Hair_dryer
0,0.930000,0.260000,0.0,6.0,1.0,40.75362,-73.98377,2.0,1.048801,1.0,...,3.0,151.0,67.0,9.0,31.0,4532.0,4094.0,459.0,50.000000,1.0
1,0.980000,0.930000,0.0,1.0,1.0,40.68514,-73.95976,3.0,1.000000,3.0,...,8.0,165.0,35.0,6.0,35.0,4443.0,2320.0,9.0,24.333333,1.0
2,0.922288,0.575276,0.0,1.0,1.0,40.68688,-73.95596,2.0,1.000000,1.0,...,1.0,175.0,69.0,8.0,5.0,4385.0,4271.0,431.0,30.000000,0.0
3,1.000000,1.000000,0.0,1.0,0.0,40.76468,-73.98315,2.0,1.000000,1.0,...,5.0,165.0,21.0,4.0,18.0,4357.0,4293.0,133.0,39.500000,1.0
4,0.880550,0.498145,0.0,1.0,1.0,40.80178,-73.96723,1.0,1.000000,1.0,...,7.0,171.0,140.0,7.0,16.0,4384.0,4170.0,1296.0,75.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26628,1.000000,0.980000,1.0,3.0,1.0,40.67319,-73.84068,2.0,1.000000,1.0,...,2.0,168.0,1.0,10.0,16.0,1344.0,1.0,1.0,55.000000,0.0
26629,1.000000,1.000000,0.0,4.0,1.0,40.79851,-73.96004,1.0,4.000000,5.0,...,6.0,1.0,23.0,5.0,20.0,2013.0,6.0,6.0,46.000000,1.0
26630,1.000000,0.980000,0.0,4.0,1.0,40.70146,-73.94368,2.0,1.000000,1.0,...,7.0,146.0,221.0,7.0,28.0,1543.0,1.0,1.0,32.500000,1.0
26631,0.910000,1.000000,0.0,1.0,1.0,40.70430,-73.92561,2.0,1.000000,1.0,...,6.0,1.0,1.0,6.0,20.0,487.0,5.0,5.0,15.000000,1.0


Dataframe has 28 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


**Now, replace the incomplete columns in df2 with the corresponding imputed columns from df_num1**

In [14]:
df2[cols] = df_num1[cols].values

df2.head(5)

,last_scraped,name,description,host_name,host_since,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,...,has_Hair_dryer,flag,host_response_time_indicator,host_response_rate_indicator,host_acceptance_rate_indicator,bedrooms_indicator,description_indicator,beds_indicator,bathrooms_text_indicator,name_indicator
0,2021-02-05,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",Jennifer,2008-09-09,A New Yorker since 2000! My passion is creatin...,within a few hours,0.930000,0.260000,0.0,...,1.0,0,0,0,0,1,0,0,0,0
1,2021-02-05,"Whole flr w/private bdrm, bath & kitchen(pls r...","Enjoy 500 s.f. top floor in 1899 brownstone, w...",LisaRoxanne,2008-12-07,Laid-back Native New Yorker (formerly bi-coast...,within a few hours,0.980000,0.930000,0.0,...,1.0,1,0,0,0,0,0,0,0,0
2,2021-02-05,BlissArtsSpace!,<b>The space</b><br />HELLO EVERYONE AND THANK...,Garon,2009-02-03,"I am an artist(painter, filmmaker) and curato...",NaN,0.922288,0.575276,0.0,...,0.0,0,1,1,1,0,0,0,1,0
3,2021-02-05,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,Shunichi,2009-03-03,I used to work for a financial industry but no...,within a day,1.000000,1.000000,0.0,...,1.0,1,0,0,0,0,0,0,0,0
4,2021-02-06,Cozy Clean Guest Room - Family Apt,"Our best guests are seeking a safe, clean, spa...",MaryEllen,2009-02-05,Welcome to family life with my oldest two away...,NaN,0.880550,0.498145,0.0,...,1.0,0,1,1,1,0,0,0,0,0


In [15]:
# Check Missing values statistics again
miss_values = missing_values(df2)
miss_values.head(20)

Dataframe has 52 columns.
There are 5 columns that have missing values.


,Missing Values,% of Total Values
host_response_time,13059,49.0
host_about,9559,35.9
description,459,1.7
bathrooms_text,39,0.1
name,5,0.0


### We have taken care of the numerical values, and need to work on the categorical values next...

In [16]:
df2['host_response_time'].mode()
df2.shape

0    within an hour
dtype: object

(26633, 52)

In [17]:
df2['host_about'] = df2['host_about'].fillna("Unknown")
df2['host_response_time'] = df2['host_response_time'].fillna("within an hour")

In [18]:

df2 = df2.dropna(axis=0) 

df2.shape

# Check Missing values statistics again
miss_values = missing_values(df2)
miss_values.head(20)

(26137, 52)

Dataframe has 52 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


### 3. Train test split

In [19]:
df2.select_dtypes(include=[object]).columns

Index(['last_scraped', 'name', 'description', 'host_name', 'host_since',
       'host_about', 'host_response_time', 'host_verifications',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'room_type',
       'bathrooms_text', 'amenities', 'first_review', 'last_review'],
      dtype='object')

In [22]:
df_split = df2[['host_response_rate',
       'host_acceptance_rate', 'host_is_superhost',
       'host_total_listings_count', 
       'host_identity_verified', 'latitude', 'longitude',
       'accommodates', 'bedrooms', 'beds', 
       'price', 'minimum_nights', 'maximum_nights', 'number_of_reviews',
       'number_of_reviews_l30d',
       'review_scores_rating', 'instant_bookable', 'reviews_per_month',
       'name_length', 'description_length', 'host_about_length',
       'verifications_length', 'amenities_length', 'host_since_days',
       'first_reviews_days', 'last_review_days', 'price_per_accommodates',
       'has_Hair_dryer', 'flag', 'host_response_time_indicator',
       'host_response_rate_indicator', 'host_acceptance_rate_indicator',
       'bedrooms_indicator', 'description_indicator', 'beds_indicator',
       'bathrooms_text_indicator', 'name_indicator']]

In [23]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df_split, test_size=0.2, random_state=42)

test_set.head()

,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_identity_verified,latitude,longitude,accommodates,bedrooms,beds,...,has_Hair_dryer,flag,host_response_time_indicator,host_response_rate_indicator,host_acceptance_rate_indicator,bedrooms_indicator,description_indicator,beds_indicator,bathrooms_text_indicator,name_indicator
23984,0.670000,1.000000,0.0,11.0,1.0,40.75643,-73.98647,2.0,1.0,1.0,...,1.0,1,0,0,0,0,0,0,0,0
1707,0.500000,0.830000,0.0,8.0,1.0,40.75639,-73.96558,4.0,1.0,2.0,...,1.0,1,0,0,0,0,0,0,0,0
14244,0.818969,0.687823,0.0,1.0,1.0,40.76152,-73.99897,3.0,1.0,2.0,...,1.0,0,1,1,1,0,0,0,0,0
22627,1.000000,0.890000,1.0,1.0,0.0,40.82560,-73.95187,1.0,1.0,1.0,...,1.0,1,0,0,0,0,0,0,0,0
2136,1.000000,0.970000,0.0,1.0,1.0,40.69594,-73.96715,2.0,1.0,1.0,...,1.0,1,0,0,0,0,0,0,0,0


## ML FLOW

### Attempt 1

In [31]:
import numpy as np
import pandas as pd
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
  
# Load the data
X_train = train_set.drop(columns=['price'], axis=1)
y_train = train_set['price']

X_test = test_set.drop(columns=['price'], axis=1)
y_test = test_set['price']

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
  
# Instantiation
xgb_r = xg.XGBRegressor(objective ='reg:squarederror',
                  n_estimators = 10, seed = 123)
  
# Fitting the model
xgb_r.fit(X_train, y_train)
  
# Predict the model
pred = xgb_r.predict(X_test)
  
# RMSE Computation
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))\

#ML Flow
with mlflow.start_run():
  mlflow.log_param('alpha','0.1')
  mlflow.log_metric('mae', 0.05)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=8, num_parallel_tree=1, random_state=123,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=123,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

RMSE :  85.535391


### Attempt 2 (use this in Databricks)

In [62]:
from math import exp
import xgboost as xgb
#!pip install hyperopt
from hyperopt import fmin, hp, tpe, SparkTrials, STATUS_OK
import mlflow
from sklearn.preprocessing import StandardScaler

In [63]:
def params_to_xgb(params):
  return{
    'objective': 'reg:squarederror',
    #'eval_metric': 'rmse',
    'max_depth': int(params['max_depth']),
    'learning_rate': exp(params['learning_rate']),
    'reg_alpha': exp(params['reg_alpha']),
    'reg_lambda': exp(params['reg_lambda']),
    'gamma': exp(params['gamma']),
    'min_child_weight': exp(params['min_child_weight']),
   # 'importance_type': 'total_gain',
    'seed': 0
  }

In [ ]:
#BASELINE MODEL

import mlflow
import mlflow.pyfunc
import mlflow.sklearn
import numpy as np
import xgboost as xg
from sklearn.metrics import mean_squared_error
from mlflow.models.signature import infer_signature
 
# The predict method of sklearn's RandomForestClassifier returns a binary classification (0 or 1). 
# The following code creates a wrapper function, SklearnModelWrapper, that uses 
# the predict_proba method to return the probability that the observation belongs to each class. 
 
class SklearnModelWrapper(mlflow.pyfunc.PythonModel):
  def __init__(self, model):
    self.model = model
    
  def predict(self, context, model_input):
    return self.model.predict(model_input)
 
# mlflow.start_run creates a new MLflow run to track the performance of this model. 
# Within the context, you call mlflow.log_param to keep track of the parameters used, and
# mlflow.log_metric to record metrics like accuracy.
with mlflow.start_run():
  n_estimators = 10
  model = xg.XGBRegressor(n_estimators=n_estimators, random_state=np.random.RandomState(123))
  model.fit(X_train, y_train)
 
  # predict_proba returns [prob_negative, prob_positive], so slice the output with [:, 1]
  predictions_test = model.predict(X_test)
  mse = mean_squared_error(y_test, predictions_test)
  mlflow.log_param('n_estimators', n_estimators)
  # Use the area under the ROC curve as a metric.
  mlflow.log_metric('mse', mse)
  wrappedModel = SklearnModelWrapper(model)
  # Log the model with a signature that defines the schema of the model's inputs and outputs. 
  # When the model is deployed, this signature will be used to validate inputs.
  signature = infer_signature(X_train, wrappedModel.predict(None, X_train))
  mlflow.pyfunc.log_model("random_forest_model", python_model=wrappedModel, signature=signature)

In [ ]:
#ATTEMPT

from hyperopt import fmin, tpe, hp, SparkTrials, Trials, STATUS_OK
from hyperopt.pyll import scope
from math import exp
import mlflow.xgboost
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error

search_space = {
  'max_depth': hp.quniform('max_depth', 20, 60, 1),
  'learning_rate': hp.uniform('learning_rate', -3, 0),
  'reg_alpha': hp.uniform('reg_alpha', -5, -1),
  'reg_lambda': hp.uniform('reg_lambda', -6, -1),
  'gamma': hp.uniform('gamma', -6, -1),
  'min_child_weight': hp.uniform('min_child_weight', -1, 3),
  'n_jobs': -1,
  'objective': 'reg:squarederror',
  'seed': 123,
}

X_train = train_set.drop(columns=['price'], axis=1)
y_train = train_set['price']

X_test = test_set.drop(columns=['price'], axis=1)
y_test = test_set['price']

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

def train_model(params):
  mlflow.xgboost.autolog()
  with mlflow.start_run(nested=True):
    train=xgb.DMatrix(data=X_train, label=y_train)
    test=xgb.DMatrix(data=X_test, label=y_test)
    booster = xgb.train(params=params_to_xgb(params), dtrain=train, num_boost_round=10,\
                        evals=[(test, "test"), (train, "train")], early_stopping_rounds=50)
    predictions = booster.predict(test)
    mse = mean_squared_error(y_test, predictions)
    mlflow.log_metric('mse', mse)
    mlflow.log_params(params_to_xgb(params))
  
    signature=infer_signature(X_train, booster.predict(train))
    mlflow.xgboost.log_model(booster, 'model', signature=signature)
  
    #mlflow.log_param('best_iteration', booster.attr('best_iteration'))
    return {'status':STATUS_OK, 'loss': -1*mse, 'booster':booster.attributes()}

#spark_trials = SparkTrials(parallelism=4)

with mlflow.start_run():
  best_params = fmin(fn=train_model, space=search_space, algo=tpe.suggest, max_evals=10, rstate=np.random.RandomState(123))

### Attempt 3

In [28]:
#!pip install mlflow
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    
    
    train_x = train_set.drop(columns=['price'], axis=1)
    train_y = train_set['price']

    test_x = test_set.drop(columns=['price'], axis=1)
    test_y = test_set['price']


    alpha = float(sys.argv[1]) if len(sys.argv) > 1 else 0.5
    l1_ratio = float(sys.argv[2]) if len(sys.argv) > 2 else 0.5

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry does not work with file store
        if tracking_url_type_store != "file":

            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
        else:
            mlflow.sklearn.log_model(lr, "model")

ValueError: could not convert string to float: '-f'

In [1]:
!pip install mlflow
!pip install hyperopt

In [2]:
import mlflow
import mlflow.pyfunc
import mlflow.sklearn
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope
from IPython.display import Image
import numpy as np
import lightgbm as lgb
from lightgbm import LGBMModel,LGBMClassifier
from math import exp
import mlflow.xgboost
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [3]:
search_space = {"max_depth":scope.int(hp.quniform("max_depth",2,100,5)),
                "n_estimators":scope.int(hp.quniform("n_estimators",2,100,1)),
                "num_leaves": scope.int(hp.quniform("num_leaves",2,50,1)),
                "reg_alpha": hp.loguniform('reg_li',-5,5),
                "random_state":1,
                "learning_rate": hp.loguniform("learning_rate", np.log(0.01), np.log(0.5)),
                "min_child_weight": hp.uniform('min_child_weight', 0.5, 10),
                "boosting": hp.choice("boosting",["gbdt","dart","goss"]),
                "objective":"reg:squarederror"
}

In [5]:
def train_model(parameters):
    mlflow.xgboost.autolog()
    with mlflow.start_run(nested=True):
        booster = xgb.XGBRegressor()
        booster.set_params(**parameters)
        booster.fit(X_train,y_train)
        
        mlflow.log_params(parameters)
        
        score = cross_val_score(booster, X_train, y_train, cv=5, 
                               scoring = "mse",n_jobs=-1)
        mean_score = np.mean(score)
        
        mlflow.log_metric('mse', mean_score)
        
        return{'status':STATUS_OK,
               "loss":-1*mean_score,
               'booster':booster.get_params}
    
with mlflow.start_run(run_name='airbnb'):
    best_params = fmin(
        fn=train_model,
        space=search_space,
        algo=tpe.suggest,
        max_evals = 50,
        trials = Trials(),
        rstate=np.random.RandomState(1)
    )

MissingConfigException: Yaml file 'C:\Users\e_wor\OneDrive\Documents\McGill MMA\Enterprise Data Science\Project II\Airbnb\mlruns\0\meta.yaml' does not exist.

In [ ]:
  'objective': 'reg:squarederror',
    #'eval_metric': 'rmse',
    'max_depth': int(params['max_depth']),
    'learning_rate': exp(params['learning_rate']),
    'reg_alpha': exp(params['reg_alpha']),
    'reg_lambda': exp(params['reg_lambda']),
    'gamma': exp(params['gamma']),
    'min_child_weight': exp(params['min_child_weight']),
   # 'importance_type': 'total_gain',
    'seed': 0
  }